# Alerta con Python y Sql

Import libreries

In [3]:
from datetime import datetime
import pandas as pd
import sqlalchemy as db
import numpy as np
import pyodbc

Conect with your data base

In [4]:
#Control execution time
start = datetime.now()

#Conection with your data base
conn = pyodbc.connect('Driver={Teradata Database ODBC Driver 16.20};'
                        'DBCNAME={your_db_name};'
                        'UID={user};'
                        'PWD={pasword}')

Excecute your querys with "consulta" that it is a cursor

In [5]:
period = '202309'

consulta = conn.cursor()

#Build your temporal table 
consulta.execute('''
    CREATE VOLATILE MULTISET TABLE TEMP_TC_HISTORICO_APROBADA_ALERT AS
        (
            SELECT 	Supervisor_Cd  REG_SUPERVISOR,Supervisor_Nm SUPERVIDOR,Promotor_Cd CODPROMOT,Promotor_Nm EJECUTIVO,documento_Num NRODOCTIT,Cliente_Cd,
                    CAST(AprobacionExpedADQ_Fc AS DATE) FECRESOL,MesAprobacionExpedADQ_Fc MESRESOL,CAST(EntregaTc_Fc AS DATE) FECENTREGA ,MesEntregaTc_Dsc MESENTREGA,MESACTIVACOMISION,
                    FECACTIVACOMISION,EquipoVenta_Dsc EQUIPO,SubEquipoVenta_Dsc SUBEQUIPO,EstadoSolicitud_Cd CODESTADO,Operacion_Cd CODOPERAC,Ooperacion_Dsc OPERACION
            --SELECT *
            FROM 	DLAB_DESNEGRET.CGR_TC_HISTORICO
            WHERE 	MesAprobacionExpedADQ_Fc>=(year(date)*100+month(date)-2) AND EstadoSolicitud_Cd=3 AND Operacion_Cd IN (1,2) --and NRODOCTIT = '75489595'
            AND 	Promotor_Cd IN (SELECT REG_EJECUTIVO FROM DLAB_DESNEGRET.MAESTRO_DOTACION WHERE PERIODO=(year(date)*100+month(date)) AND EQUIPO='TLV TARJETAS' 
                                AND sub_equipo = 'EMPRENDEDOR + PREFERENTE' AND PUESTO='EJECUTIVO' )
        )

    WITH DATA 
    PRIMARY INDEX(EJECUTIVO,NRODOCTIT ) ON COMMIT PRESERVE ROWS;
        ''')
    
consulta.commit()
consulta.close()
    
#Query in Teradata
query_temporal = '''
    SELECT FECRESOL, COUNT(*) CANTIDAD from TEMP_TC_HISTORICO_APROBADA_ALERT
    WHERE MESRESOL=(year(date)*100+month(date))
    AND CODESTADO=3 AND CODOPERAC IN (1,2)
    GROUP BY FECRESOL
    ORDER BY FECRESOL DESC;
'''

data_avance = pd.read_sql_query(query_temporal,conn)
df_table = data_avance.copy()

C:\Users\B41319\AppData\Local\Temp\ipykernel_5932\1403149927.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_avance = pd.read_sql_query(query_temporal,conn)


In [6]:
data_avance

,FECRESOL,CANTIDAD
0,2023-09-20,5
1,2023-09-19,4
2,2023-09-18,9
3,2023-09-16,2
4,2023-09-15,183
5,2023-09-14,236
6,2023-09-13,205
7,2023-09-12,79
8,2023-09-11,177
9,2023-09-08,219


In this part of code, i try to transform data

In [8]:
import datetime
def insert_dates(date_arr,df_original,df_dates,table,column1,column2):
    #Loop that insert data
    if table == 'activas':
        for days in date_arr:
            if days not in df_dates:
                #Define the variables for the new row
                value_column1 = days
                value_column2 = 0
                #Variable of your insert in dataframe
                new_row = {column1: value_column1,column2: value_column2}
                #Use the append method to add the new row to the DataFrame
                df_original = df_original.append(new_row, ignore_index=True)
    return df_original

#Insert of array dates
def dates_array():
    current_date = datetime.date.today() +datetime.timedelta(days=-1)
    fisrt_date = current_date.replace(day=1)

    #initialize array of dates
    dates = []

    #fisrt insert in dates
    dates.append(fisrt_date)

    while fisrt_date < current_date:
        # calculate the next day
        fisrt_date += datetime.timedelta(days=1)
        #insert the next date 
        dates.append(fisrt_date)
    return dates


#----------------------------------
#Transform data
#---------------------------------

#dates of month
date_arr = dates_array()
    
#Parameters for inserting data
df_dates_aprobadas = data_avance['FECRESOL'].to_numpy().tolist()
column1 = data_avance.columns[0]
column2 = data_avance.columns[1]

#Insert new rows
df_new_activas = insert_dates(date_arr,data_avance,df_dates_aprobadas,'activas',column1,column2)


C:\Users\B41319\AppData\Local\Temp\ipykernel_5932\1680762183.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(new_row, ignore_index=True)
C:\Users\B41319\AppData\Local\Temp\ipykernel_5932\1680762183.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(new_row, ignore_index=True)
C:\Users\B41319\AppData\Local\Temp\ipykernel_5932\1680762183.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(new_row, ignore_index=True)
C:\Users\B41319\AppData\Local\Temp\ipykernel_5932\1680762183.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_or

In [9]:
df_new_activas

,FECRESOL,CANTIDAD
0,2023-09-20,5
1,2023-09-19,4
2,2023-09-18,9
3,2023-09-16,2
4,2023-09-15,183
5,2023-09-14,236
6,2023-09-13,205
7,2023-09-12,79
8,2023-09-11,177
9,2023-09-08,219


Sort dataframe from the column "FECRESOL"

In [10]:
df_new_activas = df_new_activas.sort_values(by='FECRESOL',ascending=False)

In [11]:
df_new_activas

,FECRESOL,CANTIDAD
0,2023-09-20,5
1,2023-09-19,4
2,2023-09-18,9
19,2023-09-17,0
3,2023-09-16,2
4,2023-09-15,183
5,2023-09-14,236
6,2023-09-13,205
7,2023-09-12,79
8,2023-09-11,177


In parto of code, you insert the function that help you insert the data in the table of html

In [12]:
df_new_activas_copy = df_new_activas.copy()

# PRUBEAS

df_new_activas -> Principal

df_new_activas_copy  -> Secundario

Convertimos la data a numpy 

In [44]:
#Convert object to string data type
df_new_activas['FECRESOL'] = df_new_activas['FECRESOL'].astype(str)
df_new_activas['CANTIDAD'] = df_new_activas['CANTIDAD'].astype(str)

#df_table['FECRESOL'] = df_table['FECRESOL'].astype(str)

#Convert the dataframe to numpy
df_list_activas = df_new_activas.to_numpy().tolist()


Convertimos la segunda data a numpy

In [45]:
#Convert object to string data type
df_new_activas_copy['FECRESOL'] = df_new_activas_copy['FECRESOL'].astype(str)
df_new_activas_copy['CANTIDAD'] = df_new_activas_copy['CANTIDAD'].astype(str)

#Convert the dataframe to numpy
df_list_activas_copy = df_new_activas_copy.to_numpy().tolist()

Creamos la funcion maestra

In [47]:
def srch(df_list):
        #variables of scope function
        final=''
        b=0
        c=0
        x=0
        in_jump = '''<tr>'''
        jump_out = '''</tr>'''
        #print your array in html
        while x < len(df_list)*4:
            if b ==4:
                b=0
                c+=1
                final = final+jump_out
            if b==0:
                final = final+in_jump
            data_col = '''<td style ="border-style: groove;">'''+df_list[c][b]+'''</td>'''
            temp = data_col
            final +=data_col
            b+=1
            x+=1
        final = final+jump_out
        return final

Combinamos los 2 array para realizar la insercion de los datos

In [48]:
# Combina los elementos de array_1 y array_2 utilizando list comprehensions
array_final = [item1 + item2 for item1, item2 in zip(df_list_activas, df_list_activas_copy)]

In [49]:
#function that return the row.
tmp_table = srch(array_final)

In [50]:
tmp_table

'<tr><td style ="border-style: groove;">2023-09-20</td><td style ="border-style: groove;">5</td><td style ="border-style: groove;">2023-09-20</td><td style ="border-style: groove;">5</td></tr><tr><td style ="border-style: groove;">2023-09-19</td><td style ="border-style: groove;">4</td><td style ="border-style: groove;">2023-09-19</td><td style ="border-style: groove;">4</td></tr><tr><td style ="border-style: groove;">2023-09-18</td><td style ="border-style: groove;">9</td><td style ="border-style: groove;">2023-09-18</td><td style ="border-style: groove;">9</td></tr><tr><td style ="border-style: groove;">2023-09-17</td><td style ="border-style: groove;">0</td><td style ="border-style: groove;">2023-09-17</td><td style ="border-style: groove;">0</td></tr><tr><td style ="border-style: groove;">2023-09-16</td><td style ="border-style: groove;">2</td><td style ="border-style: groove;">2023-09-16</td><td style ="border-style: groove;">2</td></tr><tr><td style ="border-style: groove;">2023-

PRUEBA CON 2 DATASET

In [43]:
# Combina los elementos de array_1 y array_2 utilizando list comprehensions
array_final = [item1 + item2 for item1, item2 in zip(df_list_activas, df_list_activas_copy)]

# Imprime el resultado
print(array_final)

[['2023-09-20', '5', '2023-09-20', '5'], ['2023-09-19', '4', '2023-09-19', '4'], ['2023-09-18', '9', '2023-09-18', '9'], ['2023-09-17', '0', '2023-09-17', '0'], ['2023-09-16', '2', '2023-09-16', '2'], ['2023-09-15', '183', '2023-09-15', '183'], ['2023-09-14', '236', '2023-09-14', '236'], ['2023-09-13', '205', '2023-09-13', '205'], ['2023-09-12', '79', '2023-09-12', '79'], ['2023-09-11', '177', '2023-09-11', '177'], ['2023-09-10', '0', '2023-09-10', '0'], ['2023-09-09', '0', '2023-09-09', '0'], ['2023-09-08', '219', '2023-09-08', '219'], ['2023-09-07', '268', '2023-09-07', '268'], ['2023-09-06', '274', '2023-09-06', '274'], ['2023-09-05', '255', '2023-09-05', '255'], ['2023-09-04', '216', '2023-09-04', '216'], ['2023-09-03', '0', '2023-09-03', '0'], ['2023-09-02', '117', '2023-09-02', '117'], ['2023-09-01', '278', '2023-09-01', '278']]


In [7]:
def dates_array():
    import datetime
    current_date = datetime.date.today() +datetime.timedelta(days=-1)
    fisrt_date = current_date.replace(day=1)

    #initialize array of dates
    dates = []

    #fisrt insert in dates
    dates.append(fisrt_date)

    while fisrt_date < current_date:
        # calculate the next day
        fisrt_date += datetime.timedelta(days=1)
        #insert the next date 
        dates.append(fisrt_date)
    return dates

In [8]:
arreglo_dias = dates_array()

In [9]:
arreglo_dias

[datetime.date(2023, 9, 1),
 datetime.date(2023, 9, 2),
 datetime.date(2023, 9, 3),
 datetime.date(2023, 9, 4),
 datetime.date(2023, 9, 5),
 datetime.date(2023, 9, 6),
 datetime.date(2023, 9, 7),
 datetime.date(2023, 9, 8),
 datetime.date(2023, 9, 9),
 datetime.date(2023, 9, 10),
 datetime.date(2023, 9, 11),
 datetime.date(2023, 9, 12),
 datetime.date(2023, 9, 13),
 datetime.date(2023, 9, 14),
 datetime.date(2023, 9, 15),
 datetime.date(2023, 9, 16),
 datetime.date(2023, 9, 17),
 datetime.date(2023, 9, 18),
 datetime.date(2023, 9, 19),
 datetime.date(2023, 9, 20),
 datetime.date(2023, 9, 21),
 datetime.date(2023, 9, 22),
 datetime.date(2023, 9, 23),
 datetime.date(2023, 9, 24),
 datetime.date(2023, 9, 25)]